In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import svm
from sklearn.metrics import accuracy_score, precision_score, roc_auc_score, roc_curve, auc, confusion_matrix

%matplotlib inline

### Data preprocessing

In [81]:
%run Data_preprocessing_realmean跑这个.py

(15535, 62) (1225, 62)
(15535, 62) (15535, 62)
MAE: 0.85087
Config: {'alpha': 0.0}
************************************************************
There are 6 set of X
X_train_selected_t1, X_train_selected_t2, X_valid_selected_t1,X_valid_selected_t2,X_test_selected_t1,X_test_selected_t2
------------------------------------------------------------
Normalized version
X_train_selected_t1_norm, X_train_selected_t2_norm, X_valid_selected_t1_norm,X_valid_selected_t2_norm,X_test_selected_t1_norm,X_test_selected_t2_norm
------------------------------------------------------------
There are 4 set of Y
y_train_t1, y_train_t2, y_valid_t1, y_valid_t2
when training, please use: 'y_train_t1_value,y_train_t2_value,y_valid_t1_value,y_valid_t2_value'
************************************************************


In [96]:
# For training the model
X_train_realmean = pd.read_csv("../X/cor_train.csv")

# For cross validation
X_valid_realmean = pd.read_csv("../X/cor_valid.csv")

# For prediction
X_test_realmean = pd.read_csv("../X/cor_test.csv")

In [ ]:
y_train_t2 = pd.read_csv("../Task2/Y_train.csv")
y_valid_t2 = pd.read_csv("../Task2/Y_valid.csv")

## Imbalanced for train ##

In [84]:
Full = pd.DataFrame(np.concatenate((X_train_realmean,pd.DataFrame(y_train_t1_value)),axis=1))
Full

label0 = Full[Full[74]==0]
label1 = Full[Full[74]==1]
print(label0.shape,label1.shape)

(15535, 75) (1225, 75)


In [85]:
from imblearn.over_sampling import RandomOverSampler
oversample = RandomOverSampler()
x_over, y_over = oversample.fit_resample(X_train_realmean, y_train_t1_value)

y_over = pd.DataFrame(y_over)
y_over.shape
x_over.shape

(31070, 74)

In [86]:
Full = pd.DataFrame(np.concatenate((x_over,y_over),axis=1))
Full

label0 = Full[Full[74]==0]
label1 = Full[Full[74]==1]
print(label0.shape,label1.shape)

(15535, 75) (15535, 75)


In [87]:
X_train_t1_balanced=x_over

y_train_t1_balanced = y_over

In [88]:
X_train_t1_balanced

,alanine aminotransferase,albumin,alkaline phosphate,anion gap,asparate aminotransferase,basophils,bicarbonate,bilirubin,blood urea nitrogen,calcium,...,systemic vascular resistance,systolic blood pressure,temperature,tidal volume observed,tidal volume set,tidal volume spontaneous,troponin-t,weight,white blood cell count,white blood cell count urine
0,-0.254460,-1.979855,-0.318615,0.931458,-0.226618,0.000000,-1.694872,-0.298127,0.792807,-1.004507,...,0.615623,-0.638602,-0.002273,0.800876,0.440628,0.000000,0.000000,1.026004,0.634186,0.160067
1,-0.256599,-0.251806,-0.422405,1.723627,-0.254291,-0.742403,-1.444821,-0.444605,1.737977,-0.016061,...,0.000000,1.537514,-0.518339,0.000000,0.000000,0.000000,0.000000,0.000000,-0.181589,0.000000
2,-0.269432,0.000000,-0.367050,-0.330641,-0.271686,2.097036,1.019964,-0.395779,-0.439621,0.636039,...,0.000000,1.723172,0.101098,0.801063,1.039571,0.000000,0.000000,0.000000,-0.181589,0.000000
3,0.000000,0.000000,0.000000,-0.088963,0.000000,0.000000,0.484141,0.000000,-0.439621,0.910607,...,0.000000,-0.661305,-0.188482,0.000000,0.000000,0.000000,0.000000,0.000000,-0.139623,0.000000
4,0.000000,0.000000,0.000000,1.240269,0.000000,0.000000,-1.694872,0.000000,0.348021,0.224186,...,0.000000,0.603034,-0.157922,0.476344,0.680205,-1.333059,-0.317391,0.000000,-0.414731,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31065,-0.254460,-1.547843,-0.623066,0.611905,-0.177596,0.290121,-1.080461,-0.420192,-0.087498,-1.406063,...,2.895421,-1.282359,-0.439699,-0.366735,-0.397892,0.000000,0.000000,0.000000,0.081395,0.000000
31066,-0.261411,0.000000,0.508246,0.273555,-0.244012,0.000000,1.770116,-0.371366,-0.045800,0.636039,...,0.000000,0.628540,-0.788833,0.000000,0.000000,0.000000,0.000000,0.000000,0.461883,0.000000
31067,-0.256599,2.340268,-0.076438,0.394394,-0.258244,0.000000,1.234293,-0.261507,-0.717612,0.841965,...,0.000000,1.289227,0.298096,-0.626601,-0.697364,0.000000,-0.350044,-0.860060,-0.405405,0.000000
31068,0.000000,0.000000,0.000000,-1.297356,0.000000,0.000000,1.877280,0.000000,-0.045800,0.567397,...,0.000000,-0.633519,0.240369,-0.382720,0.000000,0.000000,0.000000,-0.559017,-0.172263,0.000000


## Formal Model ##

In [89]:
clf_rbf = svm.SVC(C=0.1, kernel='rbf', degree=3,probability=True)
clf_rbf.fit(X_train_t1_balanced,y_train_t1_balanced)

SVC(C=0.1, probability=True)

In [91]:
y_valid_score = clf_rbf.decision_function(X_valid_realmean)
# compute ROC curve and AUC for each learning rate
fpr, tpr, thresholds = roc_curve(y_valid_t1_value,y_valid_score)
roc_auc = auc(fpr, tpr)

print(roc_auc)


0.8621502183201089


In [ ]:
plt.title("C=1.1, kernel='rbf', degree=3", fontsize=18)
plt.scatter(X_test_selected_t1_norm[:, 0], X_test_selected_t1_norm[:, 1], c=pred_test_y, s=50, cmap='cool')
plot_svc_decision_function(clf)

In [97]:
pred_test_y = clf_rbf.predict_proba(X_test_realmean)
pd.DataFrame(pred_test_y).to_excel(excel_writer="C:/Users/16225/Documents/GitHub/Jebva-LOS-and-Mortality-Project-/prediction/SVMpred9.xlsx")

## GridsearchCV ##

In [29]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.svm import SVC

In [44]:
def my_custom_loss_func(X_valid_nozero,y_valid_t1_value):
    y_valid_score = grid.score(X_valid_nozero)
    # compute ROC curve and AUC for each learning rate
    fpr, tpr, thresholds = roc_curve(y_valid_t1_value,y_valid_score)
    roc_auc = auc(fpr, tpr)
    return roc_auc

score = make_scorer(my_custom_loss_func, greater_is_better=True)

In [98]:
param_grid={'kernel':['rbf'],'gamma':[1],'C':[0.1]}

grid=GridSearchCV(SVC(),param_grid,refit=True,verbose=2)
grid.fit(X_train_t1_balanced,y_train_t1_balanced)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time= 1.1min
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time= 1.0min
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=  56.7s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=  56.8s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=  59.4s


GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1], 'gamma': [1], 'kernel': ['rbf']},
             verbose=2)

In [99]:
print('best_params_:',grid.best_params_)
print('best_score_:',grid.best_score_)

best_params_: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
best_score_: 0.847473447055037


In [100]:
y_valid_score = clf_rbf.decision_function(X_valid_realmean)
# compute ROC curve and AUC for each learning rate
fpr, tpr, thresholds = roc_curve(y_valid_t1_value,y_valid_score)
roc_auc = auc(fpr, tpr)

print(roc_auc)

0.8621502183201089


In [20]:
clf_best = svm.SVC(C=0.1, kernel='rbf',gamma=1,probability=True)
clf_best.fit(X_train_t1_balanced,y_train_t1_balanced)
pred_test_y = clf_best.predict_proba(X_test_selected_t1_norm)

In [21]:
pred_test_y

array([[9.99999542e-01, 4.58463280e-07],
       [9.99999542e-01, 4.57604989e-07],
       [9.99999701e-01, 2.98611109e-07],
       ...,
       [9.99999553e-01, 4.47217754e-07],
       [9.99999554e-01, 4.45563885e-07],
       [9.99999542e-01, 4.58471157e-07]])

In [22]:
param_grid={'kernel':['rbf'],'gamma':[0.1,0.2,0.3,0.4,0.5],'C':[1]}

grid=GridSearchCV(SVC(),param_grid,refit=True,verbose=2,scoring='roc_auc')
grid.fit(X_train_t1_balanced,y_train_t1_balanced)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] END .........................C=1, gamma=0.1, kernel=rbf; total time=  41.0s
[CV] END .........................C=1, gamma=0.1, kernel=rbf; total time=  40.6s
[CV] END .........................C=1, gamma=0.1, kernel=rbf; total time=  41.0s
[CV] END .........................C=1, gamma=0.1, kernel=rbf; total time=  41.5s
[CV] END .........................C=1, gamma=0.1, kernel=rbf; total time=  42.2s
[CV] END .........................C=1, gamma=0.2, kernel=rbf; total time=  48.8s
[CV] END .........................C=1, gamma=0.2, kernel=rbf; total time=  49.1s
[CV] END .........................C=1, gamma=0.2, kernel=rbf; total time=  48.5s
[CV] END .........................C=1, gamma=0.2, kernel=rbf; total time=  48.4s
[CV] END .........................C=1, gamma=0.2, kernel=rbf; total time=  50.4s
[CV] END .........................C=1, gamma=0.3, kernel=rbf; total time=  51.2s
[CV] END .........................C=1, gamma=0.3,

GridSearchCV(estimator=SVC(),
             param_grid={'C': [1], 'gamma': [0.1, 0.2, 0.3, 0.4, 0.5],
                         'kernel': ['rbf']},
             scoring='roc_auc', verbose=2)

In [23]:
print('best_params_:',grid.best_params_)
print('best_score_:',grid.best_score_)

best_params_: {'C': 1, 'gamma': 0.2, 'kernel': 'rbf'}
best_score_: 1.0


In [24]:
clf_best = svm.SVC(C=1, kernel='rbf',gamma=0.2,probability=True)
clf_best.fit(X_train_t1_balanced,y_train_t1_balanced)
pred_test_y = clf_best.predict_proba(X_test_selected_t1_norm)

In [25]:
pred_test_y

array([[9.99987677e-01, 1.23233676e-05],
       [9.99994318e-01, 5.68167424e-06],
       [9.99999218e-01, 7.82283957e-07],
       ...,
       [9.99993442e-01, 6.55840056e-06],
       [9.99999301e-01, 6.99053964e-07],
       [9.99975642e-01, 2.43578876e-05]])

In [26]:
pd.DataFrame(pred_test_y).to_excel(excel_writer="C:/Users/16225/Documents/GitHub/Jebva-LOS-and-Mortality-Project-/prediction/SVMpred7.xlsx")

In [27]:
#clf_rbf = svm.SVC(C=1.1, kernel='rbf', degree=3,probability=True)
#clf_rbf.fit(X_train_t1_balanced,y_train_t1_balanced)
y_valid_score = clf_rbf.decision_function(X_valid_selected_t1_norm)
# compute ROC curve and AUC for each learning rate
fpr, tpr, thresholds = roc_curve(y_valid_t1_value,y_valid_score)
roc_auc = auc(fpr, tpr)

print(roc_auc)

0.8290911545142964


In [ ]:
clf_best_nonbalanced = svm.SVC(C=0.1, kernel='rbf',gamma=1,probability=True)
clf_best_nonbalanced.fit(X_train_selected_t1_norm,y_train_t1_value)
pred_test_y = clf_best_nonbalanced.predict_proba(X_test_selected_t1_norm)

In [ ]:
y_valid_score = clf_best_nonbalanced.decision_function(X_valid_selected_t1_norm)


# compute ROC curve and AUC for each learning rate
fpr, tpr, thresholds = roc_curve(y_valid_t1_value,y_valid_score)
roc_auc = auc(fpr, tpr)

print(roc_auc)

In [ ]:
sns.set_theme(style="whitegrid")

plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")

In [ ]:
pred_test_y

In [ ]:
pd.DataFrame(pred_test_y).to_excel(excel_writer="C:/Users/16225/Documents/GitHub/Jebva-LOS-and-Mortality-Project-/prediction/SVMpred5.xlsx")

## def GridSearchCV ##

In [ ]:
def model_fit(model: svm.SVC, 
              X: pd.DataFrame or np.array or list, 
              y: pd.DataFrame or np.array or list, 
              useTrainCV=True,
              cv_folds=5,
              early_stopping_rounds=50
              ) -> None:
    
    if useTrainCV:
        clf_param = model.get_clf_params()
        xgtrain = clf.DMatrix(X.values, label=y.values)
        cvresult = clf.cv(clf_param, 
                          xgtrain, 
                          num_boost_round=model.get_params()['n_estimators'], 
                          nfold=cv_folds,
                          metrics='auc', 
                          early_stopping_rounds=early_stopping_rounds,
                          verbose_eval=True
                          )
        model.set_params(n_estimators=cvresult.shape[0])

    # fit the algorithm on the data
    model.fit(X, y, eval_metric='auc')
    
    # predict training set
    dtrain_predictions = model.predict(X)
    dtrain_predprob = model.predict_proba(X)[:,1]
    
    # print model report
    print('*'*10)
    print("Model Report")
    print("Accuracy : %.4g" % metrics.accuracy_score(y.values, dtrain_predictions))
    print("AUC Score (Train): %f" % metrics.roc_auc_score(y, dtrain_predprob))
    print('*'*10)
    
    sns.set_style('whitegrid')
    feat_imp = (pd.Series(model.get_booster().get_fscore())
                .sort_values(ascending=False))
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')
    
    return model

In [ ]:
svm1 = svm.SVC(C=0.1, kernel='rbf',gamma=1,probability=True)

svm2 = model_fit(svm1, X_train, y_train_t1)

### RBF

In [ ]:
# select different type of kernel function and compare the score

# kernel = 'rbf'
train_rbf_scores = []
test_rbf_scores = []
auc_list=[]

for c in range(10,20,2):
    clf_rbf = svm.SVC(C=c*0.1, kernel='rbf',probability=True)
    clf_rbf.fit(X_train_selected_t1_norm,y_train_t1_value)
    y_valid_score = clf_rbf.decision_function(X_valid_selected_t1_norm)
    fpr, tpr, thresholds = roc_curve(y_valid_t1_value,y_valid_score)
    roc_auc = auc(fpr, tpr)
    
    train_rbf_scores.append(clf_rbf.score(X_train_selected_t1_norm,y_train_t1_value))
    test_rbf_scores.append(clf_rbf.score(X_valid_selected_t1_norm,y_valid_t1_value))
    auc_list.append(roc_auc)
    


plt.plot(train_rbf_scores, 'bo--')
plt.plot(test_rbf_scores, 'bo-')
plt.plot(auc_list,'bo-')
plt.legend(["rbf training score", "rbf valid score","auc"])
plt.axvline(np.argmax(train_rbf_scores), linestyle="dotted", color="red")
plt.annotate(np.max(auc_list).round(4), (np.argmax(auc_list), np.max(auc_list)), xycoords="data",
                 xytext=(40, 20), textcoords="offset pixels", arrowprops=dict(facecolor="black", shrink=0.1), fontsize=10,
                 horizontalalignment="center", verticalalignment="top")
plt.show()

In [ ]:
# select different type of kernel function and compare the score

# kernel = 'rbf'
train_rbf_scores = []
test_rbf_scores = []
rmse=[]

for c in range(10,20,2):
    clf_rbf = svm.SVR(C=c*0.1, kernel='rbf')
    clf_rbf.fit(X_train_selected_t2_norm,y_train_t2_value)
    y_valid_score = clf_rbf.score(X_valid_selected_t2_norm,y_valid_t2_value)

    
    train_rbf_scores.append(clf_rbf.score(X_train_selected_t2_norm,y_train_t2_value))
    test_rbf_scores.append(clf_rbf.score(X_valid_selected_t2_norm,y_valid_t2_value))
    rmse.append(y_valid_score)
    

plt.plot(train_rbf_scores, 'bo--')
plt.plot(test_rbf_scores, 'bo-')
plt.plot(auc_list,'bo-')
plt.legend(["rbf training score", "rbf valid score","auc"])
plt.axvline(np.argmax(train_rbf_scores), linestyle="dotted", color="red")
plt.annotate(np.max(auc_list).round(4), (np.argmax(auc_list), np.max(auc_list)), xycoords="data",
                 xytext=(40, 20), textcoords="offset pixels", arrowprops=dict(facecolor="black", shrink=0.1), fontsize=10,
                 horizontalalignment="center", verticalalignment="top")
plt.show()

In [16]:
clf_rbf = svm.SVC(C=2, kernel='rbf', degree=3,probability=True)
clf_rbf.fit(X_train_selected_t1_norm,y_train_t1_value)

KeyboardInterrupt: 

In [ ]:
pd.DataFrame(clf_rbf.predict_proba(X_test_selected_t1_norm)).to_excel(excel_writer="C:/Users/16225/Documents/GitHub/Jebva-LOS-and-Mortality-Project-/prediction/SVMpred3.xlsx")

In [ ]:
x_graph = list(range(1,10,2))
plt.plot(x_graph,train_rbf_scores, 'bo--')
plt.plot(x_graph,test_rbf_scores, 'bo-')
plt.plot(x_graph,auc_list,'bo-')
plt.xlim(1,5)
plt.legend(["rbf training score", "rbf valid score","auc"])
plt.axvline(np.argmax(train_rbf_scores), linestyle="dotted", color="red")
plt.annotate(np.max(auc_list).round(4), (np.argmax(auc_list), np.max(auc_list)), xycoords="data",
                 xytext=(40, 20), textcoords="offset pixels", arrowprops=dict(facecolor="black", shrink=0.1), fontsize=10,
                 horizontalalignment="center", verticalalignment="top")
plt.show()

In [ ]:
x_graph = list(range(1,10,2))
x_graph

In [ ]:
pred_test_y = clf_rbf.predict_proba(X_test_selected_t1_norm)

In [ ]:
pred_test_y

In [ ]:
pd.DataFrame(pred_test_y).to_excel(excel_writer="C:/Users/16225/Documents/GitHub/Jebva-LOS-and-Mortality-Project-/prediction/SVMpred6degree=3trainbalanced.xlsx")

In [ ]:
X_test_selected_t1_norm[:, 0]

In [ ]:
plt.title("C=1.1, kernel='rbf', degree=3", fontsize=18)
plt.scatter(X_test_selected_t1_norm[:, 0], X_test_selected_t1_norm[:, 1], c=pred_test_y, s=50, cmap='cool')
plot_svc_decision_function(clf)

In [ ]:
# plot the graph of ROC curve
sns.set_theme(style="whitegrid")

plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")

In [ ]:
pd.DataFrame(predictpos1).to_excel(excel_writer="C:/Users/16225/Documents/GitHub/Jebva-LOS-and-Mortality-Project-/prediction/SVMpred2.xlsx")

### poly degree=3

In [ ]:
# select different type of kernel function and compare the score

# kernel = 'poly'
train_poly_scores = []
test_poly_scores = []

for c in range(1,25,2):
    clf_poly = svm.SVC(C=c*0.1, kernel='poly', degree=3)
    clf_poly.fit(X_train_selected_t1_norm,y_train_t1_value)
    train_poly_scores.append(clf_poly.score(X_train_selected_t1_norm,y_train_t1_value))
    test_poly_scores.append(clf_poly.score(X_valid_selected_t1_norm,y_valid_t1_value))

plt.plot(train_poly_scores, 'bo--')
plt.plot(test_poly_scores, 'bo-')
plt.legend(["poly training score", "poly valid score"])
plt.axvline(np.argmax(train_poly_scores), linestyle="dotted", color="red")
plt.annotate(np.max(test_poly_scores).round(4), (np.argmax(test_poly_scores), np.max(test_poly_scores)), xycoords="data",
                 xytext=(40, 20), textcoords="offset pixels", arrowprops=dict(facecolor="black", shrink=0.1), fontsize=10,
                 horizontalalignment="center", verticalalignment="top")
plt.show()

In [ ]:
clf_poly = svm.SVC(C=0.1, kernel='poly', degree=3,probability=True)
clf_poly.fit(X_train_selected_t1_norm,y_train_t1_value)
y_valid_score = clf_poly.decision_function(X_valid_selected_t1_norm)


# compute ROC curve and AUC for each learning rate
fpr, tpr, thresholds = roc_curve(y_valid_t1_value,y_valid_score)
roc_auc = auc(fpr, tpr)

# plot the graph of ROC curve
sns.set_theme(style="whitegrid")
for i in range(len(y_valid_pred)):
    y_valid_pred_true.append(y_valid_pred[i][0])
y_valid_pred_true
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")

### poly degree=2

In [ ]:
# select different type of kernel function and compare the score

# kernel = 'poly'
train_poly_scores = []
test_poly_scores = []

for c in range(1,25,2):
    clf_poly = svm.SVC(C=c*0.1, kernel='poly', degree=2)
    clf_poly.fit(X_train_selected_t1_norm,y_train_t1_value)
    train_poly_scores.append(clf_poly.score(X_train_selected_t1_norm,y_train_t1_value))
    test_poly_scores.append(clf_poly.score(X_valid_selected_t1_norm,y_valid_t1_value))

plt.plot(train_poly_scores, 'bo--')
plt.plot(test_poly_scores, 'bo-')
plt.legend(["poly training score", "poly valid score"])
plt.axvline(np.argmax(train_poly_scores), linestyle="dotted", color="red")
plt.annotate(np.max(test_poly_scores).round(4), (np.argmax(test_poly_scores), np.max(test_poly_scores)), xycoords="data",
                 xytext=(40, 20), textcoords="offset pixels", arrowprops=dict(facecolor="black", shrink=0.1), fontsize=10,
                 horizontalalignment="center", verticalalignment="top")
plt.show()

In [ ]:
clf_poly = svm.SVC(C=0.1, kernel='poly', degree=2,probability=True)
clf_poly.fit(X_train_selected_t1_norm,y_train_t1_value)
y_valid_score = clf_poly.decision_function(X_valid_selected_t1_norm)


# compute ROC curve and AUC for each learning rate
fpr, tpr, thresholds = roc_curve(y_valid_t1_value,y_valid_score)
roc_auc = auc(fpr, tpr)

# plot the graph of ROC curve
sns.set_theme(style="whitegrid")
for i in range(len(y_valid_pred)):
    y_valid_pred_true.append(y_valid_pred[i][0])
y_valid_pred_true
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")

### poly degree=4

In [ ]:
# select different type of kernel function and compare the score

# kernel = 'poly'
train_poly_scores = []
test_poly_scores = []

for c in range(1,25,2):
    clf_poly = svm.SVC(C=c*0.1, kernel='poly', degree=4)
    clf_poly.fit(X_train_selected_t1_norm,y_train_t1_value)
    train_poly_scores.append(clf_poly.score(X_train_selected_t1_norm,y_train_t1_value))
    test_poly_scores.append(clf_poly.score(X_valid_selected_t1_norm,y_valid_t1_value))

plt.plot(train_poly_scores, 'bo--')
plt.plot(test_poly_scores, 'bo-')
plt.legend(["poly training score", "poly valid score"])
plt.axvline(np.argmax(train_poly_scores), linestyle="dotted", color="red")
plt.annotate(np.max(test_poly_scores).round(4), (np.argmax(test_poly_scores), np.max(test_poly_scores)), xycoords="data",
                 xytext=(40, 20), textcoords="offset pixels", arrowprops=dict(facecolor="black", shrink=0.1), fontsize=10,
                 horizontalalignment="center", verticalalignment="top")
plt.show()

In [ ]:
clf_poly = svm.SVC(C=0.1, kernel='poly', degree=4,probability=True)
clf_poly.fit(X_train_selected_t1_norm,y_train_t1_value)
y_valid_score = clf_poly.decision_function(X_valid_selected_t1_norm)


# compute ROC curve and AUC for each learning rate
fpr, tpr, thresholds = roc_curve(y_valid_t1_value,y_valid_score)
roc_auc = auc(fpr, tpr)

# plot the graph of ROC curve
sns.set_theme(style="whitegrid")
for i in range(len(y_valid_pred)):
    y_valid_pred_true.append(y_valid_pred[i][0])
y_valid_pred_true
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")

### sigmoid

In [ ]:
# select different type of kernel function and compare the score

# kernel = 'sigmoid'
train_sigmoid_scores = []
test_sigmoid_scores = []

for c in range(1,11,2):
    clf_sigmoid = svm.SVC(C=c*0.01, kernel='sigmoid')
    clf_sigmoid.fit(X_train_selected_t1_norm,y_train_t1_value)
    train_sigmoid_scores.append(clf_sigmoid.score(X_train_selected_t1_norm,y_train_t1_value))
    test_sigmoid_scores.append(clf_sigmoid.score(X_valid_selected_t1_norm,y_valid_t1_value))



In [ ]:
plt.plot(train_sigmoid_scores, 'bo--')
plt.plot(test_sigmoid_scores, 'bo-')
plt.legend(["sigmoid training score", "sigmoid valid score"])
plt.axis([0.01, 0.11, 0.92, 0.94])
plt.axvline(np.argmax(train_sigmoid_scores), linestyle="dotted", color="red")
plt.annotate(np.max(test_sigmoid_scores).round(4), (np.argmax(test_sigmoid_scores), np.max(test_sigmoid_scores)), xycoords="data",
                 xytext=(40, 20), textcoords="offset pixels", arrowprops=dict(facecolor="black", shrink=0.1), fontsize=10,
                 horizontalalignment="center", verticalalignment="top")
plt.show()

In [ ]:
clf_sigmoid = svm.SVC(C=0.01, kernel='sigmoid',probability=True)
clf_sigmoid.fit(X_train_selected_t1_norm,y_train_t1_value)
y_valid_score = clf_sigmoid.decision_function(X_valid_selected_t1_norm)


# compute ROC curve and AUC for each learning rate
fpr, tpr, thresholds = roc_curve(y_valid_t1_value,y_valid_score)
roc_auc = auc(fpr, tpr)

# plot the graph of ROC curve
sns.set_theme(style="whitegrid")

plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")